# MHD matrix Builder.

This code computes the Modified Hausdorff Distance matrix to be used for the cluster analysis.

Version of September 25th, 2024.

## Compute of MHD matrix by blocks

### Definition of functions

In [ ]:
import os
import time
from PIL import Image
import numpy as np
import torch

In [ ]:
# Función para cargar imágenes y convertirlas en puntos
def load_images_to_points(directory, skip):
    image_files = [f for f in os.listdir(directory) if f.endswith('.png') or f.endswith('.jpg')]
    sets_of_points = []

    for image_file in image_files:
        image_path = os.path.join(directory, image_file)
        image = Image.open(image_path).convert('L')
        image_np = np.array(image) / 255.0  # Normalizar a [0, 1]

        # Binarizar la imagen
        points = np.column_stack(np.where(image_np > 0.5))
        points = points[::skip]
        sets_of_points.append(points)

    print("Listo con los puntos")
    return sets_of_points

# Función para calcular la distancia de Hausdorff media (MHD) entre dos conjuntos de puntos
def calculate_mhd_batch(tensor1, tensor2, device):
    A = tensor1.to(device)
    B = tensor2.to(device)
    if A.size(1) != B.size(1):
        raise ValueError("Ambos grupos de puntos tienen diferentes dimensiones.")

    dist_matrix = torch.cdist(A.unsqueeze(0), B.unsqueeze(0)).squeeze(0)
    fhd = torch.mean(torch.min(dist_matrix, dim=1)[0])
    rhd = torch.mean(torch.min(dist_matrix, dim=0)[0])
    mhd = max(fhd.item(), rhd.item())

    return mhd

# Función para computar y guardar un bloque de la matriz de distancias
def compute_and_save_distance_matrix_block(sets_of_points, block_start, block_size, save_dir, device):
    n = len(sets_of_points)
    end = min(block_start + block_size, n)
    block_height = end - block_start

    # Convertir los sets de puntos a tensores
    tensors = [torch.tensor(points, dtype=torch.float32) for points in sets_of_points]

    # Crear submatriz de distancias con NaN
    distance_matrix_block = np.full((block_height, n), np.nan)

    for i in range(block_height):
        tensor1 = tensors[block_start + i]
        distances = []
        for j in range(n):
            if j >= block_start + i:
                tensor2 = tensors[j]
                mhd = calculate_mhd_batch(tensor1, tensor2, device)
                distances.append(mhd)
            else:
                distances.append(np.nan)

        distance_matrix_block[i, :] = distances

        # Imprimir mensaje cada cinco líneas procesadas
        if (i + 1) % 5 == 0 or i + 1 == block_height:
            print(f"Procesadas las líneas {block_start + i - 4} a {block_start + i}")

    # Guardar la submatriz de distancias en el disco
    block_filename = f'MHD_Matrix_Block_{block_start}_{end}.npy'
    np.save(os.path.join(save_dir, block_filename), distance_matrix_block)

# Función principal adaptada para Jupyter Lab
def main(directory, start_block=0, skip=0, save_dir="C:", block_size = 2000):
    total_start_time = time.time()

    # Cargar imágenes y convertirlas en conjuntos de puntos
    sets_of_points = load_images_to_points(directory, skip)
    n = len(sets_of_points)
    # block_size = 2000  # Ajusta el tamaño del bloque según la memoria disponible

    # Crear directorio para guardar los bloques de la matriz de distancias
    # save_dir = 'Z:/data/test/blocks'  # Directorio unificado
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Seleccionar dispositivo (GPU o CPU)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Usando el dispositivo: {device}")

    for block_start in range(start_block, n, block_size):
        block_start_time = time.time()
        compute_and_save_distance_matrix_block(sets_of_points, block_start, block_size, save_dir, device)
        block_end_time = time.time()
        print(f"Procesado el bloque {block_start} a {min(block_start + block_size, n)} en {block_end_time - block_start_time:.2f} segundos")

    total_end_time = time.time()
    elapsed_time = total_end_time - total_start_time

    print(f"Todos los bloques de la matriz de distancias han sido guardados en '{save_dir}'")
    print(f"Tiempo total de ejecución: {elapsed_time:.2f} segundos")

In [ ]:
# Ejecutar la función principal
directory_edges = "Z:/data/test"
directory_save_blocks = 'Z:/data/test/blocks-test'
start_block = 0
block_size = 2
skip_points = 2
main(directory_edges, start_block, skip_points, directory_save_blocks, block_size)

---
## Putting all blocks together into one Distance Matrix

In [ ]:
import numpy as np
import os
import re

# Función para extraer los índices de inicio y fin del nombre del archivo
def extract_indices(filename):
    """Extrae los índices de inicio y fin del bloque de filas del nombre del archivo."""
    # Ajustar la expresión regular para que coincida con los patrones de nombres
    match = re.search(r'_(\d+)_(\d+)\.npy$', filename)
    if match:
        start_index = int(match.group(1))
        end_index = int(match.group(2))
        return start_index, end_index
    return None, None

# Función principal para reensamblar la matriz de distancias a partir de bloques
def assemble_distance_matrix(directory):
    # Obtener la lista de archivos en el directorio especificado
    files = [f for f in os.listdir(directory) if f.endswith('.npy')]
    
    # Filtrar los archivos que coinciden con el patrón esperado
    files = [f for f in files if extract_indices(f)[0] is not None]

    # Verificar si hay al menos un bloque
    if not files:
        print("No se encontraron bloques de matriz en el directorio especificado.")
        return

    # Ordenar los archivos en función del índice de inicio del bloque de filas
    files.sort(key=lambda f: extract_indices(f)[0])

    # Lista para almacenar las partes de la matriz
    matrix_parts = []

    # Cargar y agregar las partes de la matriz a la lista
    for file in files:
        filepath = os.path.join(directory, file)
        print(f"Leyendo y agregando al ensamblaje: {file}")  # Imprimir el nombre del archivo
        matrix_part = np.load(filepath).astype(np.float32)  # Convertir a float32 inmediatamente
        matrix_parts.append(matrix_part)

    # Reconstituir la matriz original cuadrada concatenando las partes verticalmente
    final_matrix = np.vstack(matrix_parts)

    # Copiar la parte superior triangular a la parte inferior triangular, fila por fila
    n = final_matrix.shape[0]

    for i in range(n):
        if i % 100 == 0:
            print(f"Procesando fila {i} de {n}")
        # Obtener los índices donde la fila tiene valores numéricos (no NaN)
        valid_indices = np.where(~np.isnan(final_matrix[i, :]))[0]
        for j in valid_indices:
            if i != j:
                final_matrix[j, i] = final_matrix[i, j]

    # Verificar si la matriz es cuadrada
    if final_matrix.shape[0] == final_matrix.shape[1]:
        # Guardar la matriz reconstituida en el mismo directorio
        output_path = os.path.join(directory, "final_distance_matrix.npy")
        np.save(output_path, final_matrix)
        print(f"Matriz reconstituida guardada en: {output_path}")
    else:
        print("Error: La matriz reconstituida no es cuadrada.")

# Uso en Jupyter Lab: simplemente llama a la función con el directorio deseado
# Reemplaza 'Z:/data/test/blocks' con el directorio donde se guardaron los bloques
directory = 'Z:/data/test/blocks-test'
assemble_distance_matrix(directory)
